# 2: Income Effect on Motorization

Model 2 posits that the effect of income (relative to drive alone) is the same 
for both shared ride modes and transit but is different for the other modes. 
This is represented in the model by constraining the income coefficients in 
both shared ride modes and the transit mode to be equal. (pp. 108)

In [ ]:
import larch as lx

In [ ]:
lx.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = lx.examples.MTC(format="dataset")
d

In [ ]:
m = lx.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

m.utility_co[2] = P("ASC_SR2") + P("hhinc#Moto") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#Moto") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#Moto") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = PX("tottime") + PX("totcost")

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 2, Motorized"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True)
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

We can use the `ordering` attribute to fix the ordering of parameters
and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        "totcost",
        "tottime",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
    (
        "Income",
        "hhinc.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
from pytest import approx

revealed_x = dict(zip(m.pnames, result.x))

expected_x = {
    "ASC_BIKE": -2.392930557196684,
    "ASC_SR2": -2.136886688063615,
    "ASC_SR3P": -3.531993039757641,
    "ASC_TRAN": -0.7992043160144542,
    "ASC_WALK": -0.22856821987823234,
    "hhinc#5": -0.01244760898059053,
    "hhinc#6": -0.009254876762343772,
    "hhinc#Moto": -0.002868353784483784,
    "totcost": -0.004900625909641892,
    "tottime": -0.0514322471177999,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )